## Using a Pretrained Network: Feature Extraction

Convnets used for image classification comprise of two parts: a series of convolutional and pooling layers followed by a densely connected classifier layer. The first part is called the _convolutional base_ of the network. In the context of convolutional networks, _feature extraction_ using a pre-trained network consists of using the convolutional base of this pre-trained network and passing new data through it and then feeding its output to a new, as yet untrained, convnet. 

In our case, we will pass the images of cats and dogs through the convolutional base of the VGG Network and then use its output as input for our own network that is specifically being trained to distinguish dogs from cats. One can think of this process as training the new convnet to learn the image representations of the pre-trained network. The classifier of the pre-trained network is specific to the classification problem at hand. In the case of VGG, its classifier is trained on 1000 different classes. The weights information of the classifier itself is not very useful while training a new model to distinguish dogs from cats. 